# Projeto 1 - Ciência dos Dados

Nome: Matheus Souza

Nome: Pedro Ivo

Nome: Thomas Chabro

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [58]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Para a separação de emojis e palavras:
!pip install emoji

import functools
import operator
import re
import emoji

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Lieselotte Chabro\Desktop\Insper\Arquivos VS CODE\Biblioteca\2º Semestre\CDADOS\Projeto 1 final


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'certified lover boy 2.xlsx'

In [4]:
train = pd.read_excel(filename)
train

,Treinamento,Relevância
0,mas então quer dizer que o drake foi mais um a...,1
1,🔊 atualizações spotify: \n\n• drake teve a mel...,0
2,certified lover boy vai ser agora,1
3,começando a ouvir só agora o novo álbum do dra...,1
4,"hmmmm, como assim todos são certified lover bo...",1
...,...,...
545,obrigado drake por ter lançado essa fucking ob...,3
546,meu album: certified lover boy \nmúsica favori...,3
547,"nem donda, nem certified lover boy. daí a cesa...",2
548,"o álbum “certified lover boy”, de drake, se to...",0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test

,Teste,Relevância,Classificador
0,oq falar de certified lover boy? puta álbum do...,3,NaN
1,"certified lover boy é bom, hein?!",3,NaN
2,a qualidade de donda é inversamente proporcion...,2,NaN
3,'certified lover boy' foi para o primeiro luga...,0,NaN
4,creditado em donda e certified lover boy https...,1,NaN
...,...,...,...
195,dei uma atenção maior pro certified lover boy ...,3,NaN
196,ouvindo o certified lover boy,1,NaN
197,donda e certified lover boy 🛐🛐 os cara erra nunca,3,NaN
198,"o j.cole já dropou ""the off season""\no young t...",0,NaN


___
## Classificador automático de sentimento


Para a classificação dos tweets, foram criadas quatro categorias:

**0 - NEUTRO**: Indica notícias

**1 - IRRELEVANTE**: Tweets realmente irrelevantes

**2 - RELEVANTE**: Ligado à comparação, como ocorreu frequentemente, comparando o álbum (nosso tema) com outros álbuns 

**3 - MUITO RELEVANTE**: Tweets que apresentem opiniões explícitas

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
dados = pd.read_excel('certified lover boy 2.xlsx')
dados['Relevância'].value_counts(True)

3    0.296364
1    0.287273
2    0.227273
0    0.189091
Name: Relevância, dtype: float64

In [7]:
dados_treinamento = pd.read_excel('certified lover boy 2.xlsx', 'Treinamento')
dados_teste = pd.read_excel('certified lover boy 2.xlsx', 'Teste')



In [33]:
'''
Função para separação de emojis e palavras

Esta função receberá uma string (no caso um texto), e retornará uma lista com todas as palavras e emojis separados
'''

def separador(text):
    em = text
    em_split_emoji = emoji.get_emoji_regexp().split(em)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    return em_split

In [8]:
list_special_characterers = ['@', '\n', ',', '"', '(', ')', '/', '*', '>', '<', '.', ':']

for index in dados_treinamento.index:
    new_text = ''
    for character in dados_treinamento.loc[index, 'Treinamento']:
        if character not in list_special_characterers:
            new_text += character
            dados_treinamento.loc[index, 'Treinamento'] = new_text

dados_treinamento

,Treinamento,Relevância
0,mas então quer dizer que o drake foi mais um a...,1
1,🔊 atualizações spotify • drake teve a melhor e...,0
2,certified lover boy vai ser agora,1
3,começando a ouvir só agora o novo álbum do dra...,1
4,hmmmm como assim todos são certified lover boy...,1
...,...,...
545,obrigado drake por ter lançado essa fucking ob...,3
546,meu album certified lover boy música favorita ...,3
547,nem donda nem certified lover boy daí a cesar ...,2
548,o álbum “certified lover boy” de drake se torn...,0


In [9]:
read = ''

for index in dados_treinamento.index:
    for character in dados_treinamento.loc[index, 'Treinamento']:
        read += character
        
todas_as_palavras_treinamento = read.split()

series_treinamento = pd.Series(todas_as_palavras_treinamento)

series_treinamento.value_counts()

lover             546
certified         429
boy               408
o                 297
de                259
                 ... 
sensocertified      1
fuck                1
comidas             1
máximoalô           1
posso               1
Length: 2559, dtype: int64

In [10]:
dados_treinamento['Relevância'] = dados_treinamento.Relevância.astype('category')
dados_treinamento.Relevância.cat.categories = ['Neutro', 'Irrelevante', 'Relevante', 'Muito relevante']
dados_treinamento.Relevância.cat.categories



dados_teste['Relevância'] = dados_teste.Relevância.astype('category')
dados_teste.Relevância.cat.categories = ['Neutro', 'Irrelevante', 'Relevante', 'Muito relevante']
dados_teste.Relevância.cat.categories




Index(['Neutro', 'Irrelevante', 'Relevante', 'Muito relevante'], dtype='object')

In [35]:
dados_treinamento_neutro = dados_treinamento.loc[dados_treinamento['Relevância'] == 'Neutro', :]
dados_treinamento_irrelevante = dados_treinamento.loc[dados_treinamento.Relevância == 'Irrelevante', :]
dados_treinamento_relevante = dados_treinamento.loc[dados_treinamento.Relevância == 'Relevante', :]
dados_treinamento_mt_relevante = dados_treinamento.loc[dados_treinamento.Relevância == 'Muito relevante', :]

#---------------------------------------------------------------------------------------------    

read_neutro = ''

for index in dados_treinamento_neutro.index:
    for character in dados_treinamento_neutro.loc[index, 'Treinamento']:
        read_neutro += character
    read_neutro += ' '

todas_as_palavras_treinamento_neutro = separador(read_neutro)
series_treinamento_neutro = pd.Series(todas_as_palavras_treinamento_neutro)
        
#---------------------------------------------------------------------------------------------    
    
read_irrelevante = ''

for index in dados_treinamento_irrelevante.index:
    for character in dados_treinamento_irrelevante.loc[index, 'Treinamento']:
        read_irrelevante += character
    read_irrelevante += ' '
        
todas_as_palavras_treinamento_irrelevante = separador(read_irrelevante)
series_treinamento_irrelevante = pd.Series(todas_as_palavras_treinamento_irrelevante)

#---------------------------------------------------------------------------------------------    

read_relevante = ''

for index in dados_treinamento_relevante.index:
    for character in dados_treinamento_relevante.loc[index, 'Treinamento']:
        read_relevante += character
    read_relevante += ' ' 
        
todas_as_palavras_treinamento_relevante = separador(read_relevante)
series_treinamento_relevante = pd.Series(todas_as_palavras_treinamento_relevante)         

#---------------------------------------------------------------------------------------------    

read_mt_relevante = ''

for index in dados_treinamento_mt_relevante.index:
    for character in dados_treinamento_mt_relevante.loc[index, 'Treinamento']:
        read_mt_relevante += character
    read_mt_relevante += ' '

todas_as_palavras_treinamento_mt_relevante = separador(read_mt_relevante)
series_treinamento_mt_relevante = pd.Series(todas_as_palavras_treinamento_mt_relevante)



In [36]:
# Criando tabelas com a contagem das palavras

tabela_neutro = series_treinamento_neutro.value_counts()
tabela_irrelevante = series_treinamento_irrelevante.value_counts()
tabela_relevante = series_treinamento_relevante.value_counts()
tabela_mt_relevante = series_treinamento_mt_relevante.value_counts()

# Criando então as tabelas relativas, com a contagem relativa das palavras

tabela_neutro_relativa  = series_treinamento_neutro.value_counts(True)
tabela_irrelevante_relativa  = series_treinamento_irrelevante.value_counts(True)
tabela_relevante_relativa  = series_treinamento_relevante.value_counts(True)
tabela_mt_relevante_relativa  = series_treinamento_mt_relevante.value_counts(True)



In [37]:
tabela_neutro_relativa

de           0.041801
lover        0.040997
o            0.030949
certified    0.029743
boy          0.028537
               ...   
iii          0.000402
samples      0.000402
durante      0.000402
pq           0.000402
cada         0.000402
Length: 920, dtype: float64

In [38]:
tabela_irrelevante_relativa

lover        0.075362
certified    0.072947
boy          0.068599
o            0.029952
e            0.020290
               ...   
🤰🏻           0.000483
apoiando     0.000483
dúvidas      0.000483
concretas    0.000483
vcs          0.000483
Length: 771, dtype: float64

In [39]:
tabela_relevante_relativa

lover        0.049104
certified    0.047935
boy          0.045596
donda        0.037023
o            0.033905
               ...   
concorda     0.000390
alezudoo     0.000390
thru         0.000390
então        0.000390
obg          0.000390
Length: 831, dtype: float64

In [40]:
tabela_mt_relevante_relativa

lover        0.059316
certified    0.058952
boy          0.055677
o            0.033843
de           0.027293
               ...   
foda         0.000364
mixtape      0.000364
🤤            0.000364
mid          0.000364
tocando      0.000364
Length: 825, dtype: float64

In [41]:
# Definindo as probabilidades de cada relevância

todas_as_palavras_treinamento = read_neutro + read_irrelevante + read_relevante + read_mt_relevante
todas_as_palavras_treinamento = todas_as_palavras_treinamento.split()

P_Neutro = len(todas_as_palavras_treinamento_neutro)/len(todas_as_palavras_treinamento)
P_Irrelevante = len(todas_as_palavras_treinamento_irrelevante)/len(todas_as_palavras_treinamento)
P_Relevante = len(todas_as_palavras_treinamento_relevante)/len(todas_as_palavras_treinamento)
P_Muito_Revelevante = len(todas_as_palavras_treinamento_mt_relevante)/len(todas_as_palavras_treinamento)


In [42]:
P_Neutro

0.25382574984697

In [43]:
P_Irrelevante

0.21118139155274435

In [44]:
P_Relevante

0.26178330952866763

In [45]:
P_Muito_Revelevante

0.28035094878596206

In [53]:
# Definindo função para limpar a frase, removendo caracteres especiais

def cleanup (frase):
    saida = ''
    for el in frase:
        if el not in list_special_characterers:
            saida += el
    return saida

def classificador (frase):

    frase = cleanup(frase.lower())
    lista_frase = separador(frase)
    
    # Calculando probabilidades

    Prob_ser_neutro = 1
    for palavra in lista_frase:
        if palavra not in tabela_neutro:
            x = 0
        else:
            x = tabela_neutro[palavra]
        p_palavra = (x + 1)/(len(tabela_neutro) + len(todas_as_palavras_treinamento))
        Prob_ser_neutro *= p_palavra
    Prob_ser_neutro *= P_Neutro

    Prob_ser_irrelevante = 1
    for el in lista_frase:
        if el not in tabela_irrelevante:
            x = 0
        else:
            x = tabela_irrelevante[el]
        p_el = (x + 1)/(len(tabela_irrelevante) + len(todas_as_palavras_treinamento))
        Prob_ser_irrelevante *= p_el
    Prob_ser_irrelevante *= P_Irrelevante

    Prob_ser_relevante = 1
    for el in lista_frase:
        if el not in tabela_relevante:
            x = 0
        else:
            x = tabela_relevante[el]
        p_el = (x + 1)/(len(tabela_relevante) + len(todas_as_palavras_treinamento))
        Prob_ser_relevante *= p_el
    Prob_ser_relevante *= P_Relevante

    Prob_ser_mt_relevante = 1
    for el in lista_frase:
        if el not in tabela_mt_relevante:
            x = 0
        else:
            x = tabela_mt_relevante[el]
        p_el = (x + 1)/(len(tabela_mt_relevante) + len(todas_as_palavras_treinamento))
        Prob_ser_mt_relevante *= p_el
    Prob_ser_mt_relevante *= P_Muito_Revelevante

    lista_probabilidades = [Prob_ser_neutro, Prob_ser_irrelevante, Prob_ser_relevante, Prob_ser_mt_relevante]
    i = lista_probabilidades.index(max(lista_probabilidades))
    if i == 0:
        return ('Neutro')
    if i == 1:
        return ('Irrelevante')
    if i == 2:
        return ('Relevante')
    if i == 3:
        return ('Muito relevante')

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [54]:
i = 0
for frase in dados_teste['Teste']:
    rate = classificador(frase)
    dados_teste.loc[i, 'Classificador'] = rate
    i += 1



In [55]:
dados_teste['Relevância'].value_counts()

Muito relevante    63
Neutro             56
Relevante          46
Irrelevante        35
Name: Relevância, dtype: int64

In [56]:
falso_mt_relevante = 0
verdadeiro_mt_relevante = 0
falso_relevante = 0
verdadeiro_relevante = 0
falso_irrelevante = 0
verdadeiro_irrelevante = 0
falso_neutro = 0
verdadeiro_neutro = 0




for index in dados_teste.index:
    if dados_teste.loc[index, 'Relevância'] == 'Muito relevante' and dados_teste.loc[index, 'Classificador'] == 'Muito relevante':
        verdadeiro_mt_relevante += 1
    if dados_teste.loc[index, 'Relevância'] != 'Muito relevante' and dados_teste.loc[index, 'Classificador'] == 'Muito relevante':
        falso_mt_relevante += 1
    
    if dados_teste.loc[index, 'Relevância'] == 'Relevante' and dados_teste.loc[index, 'Classificador'] == 'Relevante':
        verdadeiro_relevante += 1
    if dados_teste.loc[index, 'Relevância'] != 'Relevante' and dados_teste.loc[index, 'Classificador'] == 'Relevante':
        falso_relevante += 1
    
    if dados_teste.loc[index, 'Relevância'] == 'Irrelevante' and dados_teste.loc[index, 'Classificador'] == 'Irrelevante':
        verdadeiro_irrelevante += 1
    if dados_teste.loc[index, 'Relevância'] != 'Irrelevante' and dados_teste.loc[index, 'Classificador'] == 'Irrelevante':
        falso_irrelevante += 1
        
    if dados_teste.loc[index, 'Relevância'] == 'Neutro' and dados_teste.loc[index, 'Classificador'] == 'Neutro':
        verdadeiro_neutro += 1
    if dados_teste.loc[index, 'Relevância'] != 'Neutro' and dados_teste.loc[index, 'Classificador'] == 'Neutro':
        falso_neutro += 1
    
    
print(f'Porcentagem de verdadeiros muito relevantes: {verdadeiro_mt_relevante/(verdadeiro_mt_relevante + falso_mt_relevante)}')
print(f'Porcentagem de falsos muito relevantes: {falso_mt_relevante/(verdadeiro_mt_relevante + falso_mt_relevante)}')
print('\n')
print(f'Porcentagem de verdadeiros relevantes: {verdadeiro_relevante/(verdadeiro_relevante + falso_relevante)}')
print(f'Porcentagem de falsos relevantes: {falso_relevante/(verdadeiro_relevante + falso_relevante)}')
print('\n')
print(f'Porcentagem de verdadeiros irrelevantes: {verdadeiro_irrelevante/(verdadeiro_irrelevante + falso_irrelevante)}')
print(f'Porcentagem de falsos irrelevantes: {falso_irrelevante/(verdadeiro_irrelevante + falso_irrelevante)}')
print('\n')
print(f'Porcentagem de verdadeiros neutros: {verdadeiro_neutro/(verdadeiro_neutro + falso_neutro)}')
print(f'Porcentagem de falsos neutros: {falso_neutro/(verdadeiro_neutro + falso_neutro)}')

   
# Acurácia
acuracia = 0
total = 0
for i in dados_teste.index:
    if dados_teste.loc[i, 'Relevância'] == dados_teste.loc[i, 'Classificador']:
        acuracia += 1
    total += 1

acuracia = (acuracia/total) * 100
print('\n')
print(f'Porcentagem de acertos do Classificador (acurácia): {acuracia}')

Porcentagem de verdadeiros muito relevantes: 0.5217391304347826
Porcentagem de falsos muito relevantes: 0.4782608695652174


Porcentagem de verdadeiros relevantes: 0.5070422535211268
Porcentagem de falsos relevantes: 0.49295774647887325


Porcentagem de verdadeiros irrelevantes: 0.3
Porcentagem de falsos irrelevantes: 0.7


Porcentagem de verdadeiros neutros: 0.8518518518518519
Porcentagem de falsos neutros: 0.14814814814814814


Porcentagem de acertos do Classificador (acurácia): 55.00000000000001


___

# Concluindo

**Performance do Classificador**

Para a análise crítica do classificador, foram feitos cálculos acerca de sua performance, que darão embasamento para obter uma conclusão mais aprofundada e concreta. Entre estes cálculos, estão o cálculo da porcentagem de verdades e falsos de cada categoria, assim como a acurácia total do Classificador.

A partir da porcentagem de acertos, de aproximadamente 55%, nota-se que existe uma margem razoavelmente boa de acertos, beirando os 60%, mas ainda assim longe de um patamar considerado como elevado. Entretanto, analisar apenas este dado não nos dá um panomarama realmente completo, pois se trata de uma informação muito geral.

Dessa forma, para uma análise mais específica e, consequentemente, mais aprofundada, foram feitos os cálculos da porcetagem de verdadeiros e falsos de cada categoria. 

Analisando estes dados, nota-se que quase todas as categorias obtiveram pelo menos, aproximadamente, 50% de acertos. Podemos ainda dar ênfase para a categoria 'Muito Relevante', que se distanciou dos demais com uma porcentagem de verdadeiros de 82%. Isto pode ocorrer talvez pelo fato de a categoria 'Muito Relevante' possuir palavras mais "únicas", mais específicas da categoria. 

Por outro lado, pode-se entender estes números como um exagero por parte do Classificador, de classificar muitos tweets nesta última categoria, aumentando consequentemente o número de verdadeiros dela. Esta ideia, por sua vez, era algo esperado, dado que a categoria 'Muito Relevante' diz respeito à opiniões, que são a maioria quando falamos de twitter. 


**Outras aplicações para o classificador Naive Bayes**

O classificador Naive Bayes pode ser facilmente aplicado, dado que é relativamente rápido e necessita de poucos dados para funcionar.Dado isso, ele pode ser utilizado para diversas outras funções de classificação, incluindo previsões em tempo real.

Além da classificação de texto, utilizada nesse Projeto, podemos também aplicar, por exemplo, para a filtragem de SPAM, em e-mails. Com uma base de dados, é possível classificar quais e-mails são propaganda, canais de venda e outras mensagens indesejáveis. 

Outro exemplo muito útil, seria o diagnóstico médico automático. A partir de uma base de dados de sintomas e doenças, o classificador poderia receber os sintomas do paciente e dizer qual doença ele possui.

**Por que não utilizar o classificador para gerar mais amostras de treinamento?**

Explicou porquê não pode usar o próprio classificador para gerar mais amostras de treinamento: O classificador não é uma inteligência artificial perfeita, logo ele classifica alguns tweets com a relevância errada. Dessa forma, se usássemos os tweets do classificador como base de treinamento, o nosso naive bayes treinaria (tomando como verdade) com alguns falsos positivos e falsos negativos, alterando totalmente a classificação dos proximos tweets a serem testados.

___

# Sugestões de Aperfeiçoamento

**1. Utilizar probabilidas com log**

Umas das possibilidades de melhora do classificador, é trabalhar com as possibilidades em log. Para evitar trabalhar com números muito pequenos, podemos usar o logarítimo dos valores das probabilidades. 
A função de log mapeia os valores de probabilidade do intervalo [0, 1], fazendo com que os valores das probabilidade do log sejam negativos. Dessa forma, quanto menor for o valor do logarítimo, ele será mapeado para um valor maior negativo pela função de log.

Assim, para aplicar, deveríamos utilizar a equação:

<p><img src='https://www.baeldung.com/wp-content/ql-cache/quicklatex.com-6beadb40c93de8340c1b656900e6b548_l3.svg' alt='equacao_img'></p>

Referência: <https://www.baeldung.com/cs/naive-bayes-classification-performance>





**2. Eliminação de "stopwords**

Stopwords são palavras que não necessariamente ajudam na construção e treinamento do classificador. Artigos como 'a' e 'o', ou outras palavras genéricas podem ser removidas, a fim de obter apenas as "palavras chave" do texto. Uma frase como 'O novo álbum do draka é ruim', seria reduzida apenas à 'novo álbum drake ruim', ajudando o classificador por eliminar palavras sem uma real importância.

Para aplicar este método, deveria ser criada uma lista com as palavras a serem eliminadas. Tendo esta lista, seriam rodadas todas as palavras do treinamento, e então removidas as pertencentes à lista.

Entretanto, este processo pode não ser necessariamente benéfico ao classificador. Isto pois o uso de artigos e outras palavras pode ser característico de frases mais bem elaboradas, o que ajudaria o classificador à diferenciar uma frase desse tipo de outras mais simples.

Referência: <https://en.wikipedia.org/wiki/Stop_word>

**3. Lematização de palavras**

Fazer o agrupamento de diferentes palavras derivadas de uma mesma palavra, para que obtenham a mesma classificação, aperfeiçoando o classificador.

**4. Uso n-gramas**

Ao invés de contar palavras únicas, este método faz uma contagem de uma sequência de palavras específicas de sua preferência, a qual terá uma certa classificação.

**5. Uso de TF-IDF**

Fazer uma penalização para as palavras mais frequentes do dataframe. No caso desse projeto, seriam as palavras chave do tweet(No nosso, pode ser certified, lover e boy), as quais aparecem obrigatoriamente nos tweets. Isso faria com que a relevância delas não fosse considerada, melhorando a qualidade do classificador.

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**